In [1]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
for i in range(len(y_train_valid)):
    if(y_train_valid[i] == 769):
        y_train_valid[i] = 0
    if(y_train_valid[i] == 770):
        y_train_valid[i] = 1
    if(y_train_valid[i] == 771):
        y_train_valid[i] = 2
    if(y_train_valid[i] == 772):
        y_train_valid[i] = 3

In [4]:
for i in range(len(y_test)):
    if(y_test[i] == 769):
        y_test[i] = 0
    if(y_test[i] == 770):
        y_test[i] = 1
    if(y_test[i] == 771):
        y_test[i] = 2
    if(y_test[i] == 772):
        y_test[i] = 3

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datatype = torch.float
torch.cuda.device(device)
print(device)

cuda:0


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Temporal Convolution
        self.conv1 = nn.Conv2d(1, 40, (1, 40), stride = (1, 2))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(40)
        # ELU
        self.elu1 = nn.ELU()
        # Spatial Convolution
        self.conv2 = nn.Conv2d(40, 40, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(40)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(40 * 28, 4)
        # Softmax
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [8]:
cnn = CNN().to(device)

In [9]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [11]:
batch_size = 500
num_iters = 1
num_epoch = 1000
epoch_loss = [0.0 for i in range(num_epoch)]
train_accuracy = [0.0 for i in range(num_epoch)]
validation_accuracy = [0.0 for i in range(num_epoch)]
for epoch in range(num_epoch):
    counter = 0
    for train_index, valid_index in kf.split(X_train_valid):
        X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
        y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
        for it in np.arange(num_iters):
            index = np.random.choice(X_train_cur.shape[0], batch_size)
            X_batch = X_train_cur[index, :, :, :]
            y_batch = y_train_cur[index]
            optimizer.zero_grad()
            outputs = cnn(X_batch)

            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss[epoch] += loss

            counter += 1
            
        # Validation Accuracy for final K-fold
        output_val = cnn(X_valid_cur)
        prediction_val = torch.argmax(output_val, axis = 1)
        accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
        print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
        validation_accuracy[epoch] += accuracy_val
        del output_val
        del prediction_val
        del X_valid_cur
        del y_valid_cur
        del accuracy_val
        torch.cuda.empty_cache()

        # Training Accuracy for final K-fold
        output_train = cnn(X_train_cur)
        prediction_train = torch.argmax(output_train, axis = 1)
        accuracy_train = float(prediction_train.eq(y_train_cur).sum()) / float(y_train_cur.shape[0])
        train_accuracy[epoch] += accuracy_train

        del output_train
        del prediction_train
        del X_train_cur
        del y_train_cur
        del accuracy_train
        torch.cuda.empty_cache()
    
    

    epoch_loss[epoch] /= counter 
    print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))
train_accuracy = [i / 4.0 for i in train_accuracy]
validation_accuracy = [i / 4.0 for i in validation_accuracy]
# batch_size = 500
# num_iters = 1
# num_epoch = 1000
# epoch_loss = [0.0 for i in range(num_epoch)]

# for epoch in range(num_epoch):
#     counter = 0
#     for train_index, valid_index in kf.split(X_train_valid):
#         X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
#         X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
#         y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
#         y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
#         for it in np.arange(num_iters):
#             index = np.random.choice(X_train_cur.shape[0], batch_size)
#             X_batch = X_train_cur[index, :, :, :]
#             y_batch = y_train_cur[index]
#             optimizer.zero_grad()
#             outputs = cnn(X_batch)

#             loss = criterion(outputs, y_batch)
#             loss.backward()
#             optimizer.step()
#             epoch_loss[epoch] += loss

#             counter += 1
#         # Validation Accuracy for current K-fold
#         output_val = cnn(X_valid_cur)
#         prediction_val = torch.argmax(output_val, axis = 1)
#         accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
#         print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
#     epoch_loss[epoch] /= counter 
#     print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))

Current Epoch: 0, Validation Accuracy: 0.33081285444234404
Current Epoch: 0, Validation Accuracy: 0.3705103969754253
Current Epoch: 0, Validation Accuracy: 0.41965973534971646
Current Epoch: 0, Validation Accuracy: 0.38257575757575757
Current Epoch: 0, Epoch Loss: 1.3516942262649536
Current Epoch: 1, Validation Accuracy: 0.3761814744801512
Current Epoch: 1, Validation Accuracy: 0.391304347826087
Current Epoch: 1, Validation Accuracy: 0.45368620037807184
Current Epoch: 1, Validation Accuracy: 0.4071969696969697
Current Epoch: 1, Epoch Loss: 1.3102113008499146
Current Epoch: 2, Validation Accuracy: 0.43100189035916825
Current Epoch: 2, Validation Accuracy: 0.46502835538752363
Current Epoch: 2, Validation Accuracy: 0.5179584120982986
Current Epoch: 2, Validation Accuracy: 0.4621212121212121
Current Epoch: 2, Epoch Loss: 1.278108835220337
Current Epoch: 3, Validation Accuracy: 0.43667296786389415
Current Epoch: 3, Validation Accuracy: 0.5255198487712666
Current Epoch: 3, Validation Accurac

Current Epoch: 29, Validation Accuracy: 0.6767485822306238
Current Epoch: 29, Validation Accuracy: 0.7164461247637051
Current Epoch: 29, Validation Accuracy: 0.7372400756143668
Current Epoch: 29, Validation Accuracy: 0.7367424242424242
Current Epoch: 29, Epoch Loss: 1.0378868579864502
Current Epoch: 30, Validation Accuracy: 0.6748582230623819
Current Epoch: 30, Validation Accuracy: 0.7051039697542533
Current Epoch: 30, Validation Accuracy: 0.7542533081285444
Current Epoch: 30, Validation Accuracy: 0.7462121212121212
Current Epoch: 30, Epoch Loss: 1.0346508026123047
Current Epoch: 31, Validation Accuracy: 0.6956521739130435
Current Epoch: 31, Validation Accuracy: 0.722117202268431
Current Epoch: 31, Validation Accuracy: 0.7372400756143668
Current Epoch: 31, Validation Accuracy: 0.7367424242424242
Current Epoch: 31, Epoch Loss: 1.0146715641021729
Current Epoch: 32, Validation Accuracy: 0.7013232514177694
Current Epoch: 32, Validation Accuracy: 0.7315689981096408
Current Epoch: 32, Valida

Current Epoch: 57, Epoch Loss: 0.9705137014389038
Current Epoch: 58, Validation Accuracy: 0.7674858223062382
Current Epoch: 58, Validation Accuracy: 0.782608695652174
Current Epoch: 58, Validation Accuracy: 0.8147448015122873
Current Epoch: 58, Validation Accuracy: 0.8049242424242424
Current Epoch: 58, Epoch Loss: 0.9673165678977966
Current Epoch: 59, Validation Accuracy: 0.7939508506616257
Current Epoch: 59, Validation Accuracy: 0.776937618147448
Current Epoch: 59, Validation Accuracy: 0.8166351606805293
Current Epoch: 59, Validation Accuracy: 0.8162878787878788
Current Epoch: 59, Epoch Loss: 0.9666450023651123
Current Epoch: 60, Validation Accuracy: 0.7561436672967864
Current Epoch: 60, Validation Accuracy: 0.7542533081285444
Current Epoch: 60, Validation Accuracy: 0.8298676748582231
Current Epoch: 60, Validation Accuracy: 0.7954545454545454
Current Epoch: 60, Epoch Loss: 0.9555866122245789
Current Epoch: 61, Validation Accuracy: 0.7429111531190926
Current Epoch: 61, Validation Accur

Current Epoch: 86, Epoch Loss: 0.9273734092712402
Current Epoch: 87, Validation Accuracy: 0.7977315689981096
Current Epoch: 87, Validation Accuracy: 0.8241965973534972
Current Epoch: 87, Validation Accuracy: 0.8431001890359168
Current Epoch: 87, Validation Accuracy: 0.8428030303030303
Current Epoch: 87, Epoch Loss: 0.9250821471214294
Current Epoch: 88, Validation Accuracy: 0.8241965973534972
Current Epoch: 88, Validation Accuracy: 0.8204158790170132
Current Epoch: 88, Validation Accuracy: 0.8355387523629489
Current Epoch: 88, Validation Accuracy: 0.8352272727272727
Current Epoch: 88, Epoch Loss: 0.9307208061218262
Current Epoch: 89, Validation Accuracy: 0.8071833648393195
Current Epoch: 89, Validation Accuracy: 0.8147448015122873
Current Epoch: 89, Validation Accuracy: 0.8657844990548205
Current Epoch: 89, Validation Accuracy: 0.8560606060606061
Current Epoch: 89, Epoch Loss: 0.929928183555603
Current Epoch: 90, Validation Accuracy: 0.8128544423440454
Current Epoch: 90, Validation Accu

Current Epoch: 115, Validation Accuracy: 0.8752362948960303
Current Epoch: 115, Validation Accuracy: 0.8352272727272727
Current Epoch: 115, Epoch Loss: 0.9112892150878906
Current Epoch: 116, Validation Accuracy: 0.8374291115311909
Current Epoch: 116, Validation Accuracy: 0.8393194706994329
Current Epoch: 116, Validation Accuracy: 0.8544423440453687
Current Epoch: 116, Validation Accuracy: 0.8522727272727273
Current Epoch: 116, Epoch Loss: 0.9150587916374207
Current Epoch: 117, Validation Accuracy: 0.8393194706994329
Current Epoch: 117, Validation Accuracy: 0.8128544423440454
Current Epoch: 117, Validation Accuracy: 0.8185255198487713
Current Epoch: 117, Validation Accuracy: 0.8465909090909091
Current Epoch: 117, Epoch Loss: 0.9236471056938171
Current Epoch: 118, Validation Accuracy: 0.8468809073724007
Current Epoch: 118, Validation Accuracy: 0.8468809073724007
Current Epoch: 118, Validation Accuracy: 0.8487712665406427
Current Epoch: 118, Validation Accuracy: 0.8636363636363636
Current

...

Current Epoch: 879, Validation Accuracy: 0.9584120982986768
Current Epoch: 879, Validation Accuracy: 0.9603024574669187
Current Epoch: 879, Validation Accuracy: 0.9696969696969697
Current Epoch: 879, Epoch Loss: 0.7837700843811035
Current Epoch: 880, Validation Accuracy: 0.9603024574669187
Current Epoch: 880, Validation Accuracy: 0.9584120982986768
Current Epoch: 880, Validation Accuracy: 0.9603024574669187
Current Epoch: 880, Validation Accuracy: 0.9659090909090909
Current Epoch: 880, Epoch Loss: 0.7738412022590637
Current Epoch: 881, Validation Accuracy: 0.9546313799621928
Current Epoch: 881, Validation Accuracy: 0.9603024574669187
Current Epoch: 881, Validation Accuracy: 0.9659735349716446
Current Epoch: 881, Validation Accuracy: 0.9621212121212122
Current Epoch: 881, Epoch Loss: 0.7737367153167725
Current Epoch: 882, Validation Accuracy: 0.9603024574669187
Current Epoch: 882, Validation Accuracy: 0.9697542533081286
Current Epoch: 882, Validation Accuracy: 0.9565217391304348
Current

Current Epoch: 907, Validation Accuracy: 0.9603024574669187
Current Epoch: 907, Validation Accuracy: 0.9564393939393939
Current Epoch: 907, Epoch Loss: 0.7666369080543518
Current Epoch: 908, Validation Accuracy: 0.9584120982986768
Current Epoch: 908, Validation Accuracy: 0.9716446124763705
Current Epoch: 908, Validation Accuracy: 0.9527410207939508
Current Epoch: 908, Validation Accuracy: 0.9696969696969697
Current Epoch: 908, Epoch Loss: 0.7825050354003906
Current Epoch: 909, Validation Accuracy: 0.9603024574669187
Current Epoch: 909, Validation Accuracy: 0.9584120982986768
Current Epoch: 909, Validation Accuracy: 0.9659735349716446
Current Epoch: 909, Validation Accuracy: 0.9621212121212122
Current Epoch: 909, Epoch Loss: 0.7785698175430298
Current Epoch: 910, Validation Accuracy: 0.9603024574669187
Current Epoch: 910, Validation Accuracy: 0.9697542533081286
Current Epoch: 910, Validation Accuracy: 0.9697542533081286
Current Epoch: 910, Validation Accuracy: 0.9621212121212122
Current

Current Epoch: 935, Validation Accuracy: 0.9734848484848485
Current Epoch: 935, Epoch Loss: 0.7756983041763306
Current Epoch: 936, Validation Accuracy: 0.9584120982986768
Current Epoch: 936, Validation Accuracy: 0.9678638941398866
Current Epoch: 936, Validation Accuracy: 0.9735349716446124
Current Epoch: 936, Validation Accuracy: 0.9734848484848485
Current Epoch: 936, Epoch Loss: 0.7685196399688721
Current Epoch: 937, Validation Accuracy: 0.9621928166351607
Current Epoch: 937, Validation Accuracy: 0.9735349716446124
Current Epoch: 937, Validation Accuracy: 0.9716446124763705
Current Epoch: 937, Validation Accuracy: 0.9753787878787878
Current Epoch: 937, Epoch Loss: 0.7754409313201904
Current Epoch: 938, Validation Accuracy: 0.9621928166351607
Current Epoch: 938, Validation Accuracy: 0.9659735349716446
Current Epoch: 938, Validation Accuracy: 0.9735349716446124
Current Epoch: 938, Validation Accuracy: 0.9696969696969697
Current Epoch: 938, Epoch Loss: 0.7747855186462402
Current Epoch: 9

Current Epoch: 963, Epoch Loss: 0.7726774215698242
Current Epoch: 964, Validation Accuracy: 0.9621928166351607
Current Epoch: 964, Validation Accuracy: 0.9735349716446124
Current Epoch: 964, Validation Accuracy: 0.9678638941398866
Current Epoch: 964, Validation Accuracy: 0.9734848484848485
Current Epoch: 964, Epoch Loss: 0.7771965265274048
Current Epoch: 965, Validation Accuracy: 0.9603024574669187
Current Epoch: 965, Validation Accuracy: 0.9697542533081286
Current Epoch: 965, Validation Accuracy: 0.9735349716446124
Current Epoch: 965, Validation Accuracy: 0.9715909090909091
Current Epoch: 965, Epoch Loss: 0.7764792442321777
Current Epoch: 966, Validation Accuracy: 0.9640831758034026
Current Epoch: 966, Validation Accuracy: 0.9735349716446124
Current Epoch: 966, Validation Accuracy: 0.9716446124763705
Current Epoch: 966, Validation Accuracy: 0.9640151515151515
Current Epoch: 966, Epoch Loss: 0.7778419256210327
Current Epoch: 967, Validation Accuracy: 0.9565217391304348
Current Epoch: 9

Current Epoch: 992, Validation Accuracy: 0.9603024574669187
Current Epoch: 992, Validation Accuracy: 0.9716446124763705
Current Epoch: 992, Validation Accuracy: 0.9584120982986768
Current Epoch: 992, Validation Accuracy: 0.9526515151515151
Current Epoch: 992, Epoch Loss: 0.7877981662750244
Current Epoch: 993, Validation Accuracy: 0.9546313799621928
Current Epoch: 993, Validation Accuracy: 0.9716446124763705
Current Epoch: 993, Validation Accuracy: 0.943289224952741
Current Epoch: 993, Validation Accuracy: 0.9545454545454546
Current Epoch: 993, Epoch Loss: 0.781754732131958
Current Epoch: 994, Validation Accuracy: 0.9659735349716446
Current Epoch: 994, Validation Accuracy: 0.9678638941398866
Current Epoch: 994, Validation Accuracy: 0.9621928166351607
Current Epoch: 994, Validation Accuracy: 0.9299242424242424
Current Epoch: 994, Epoch Loss: 0.7835763096809387
Current Epoch: 995, Validation Accuracy: 0.9621928166351607
Current Epoch: 995, Validation Accuracy: 0.9697542533081286
Current E

In [12]:
X_test = torch.tensor(X_test.reshape(X_test.shape[0], -1, 22, 1000), dtype= torch.float).to(device)
y_test = torch.tensor(y_test, dtype = torch.long).to(device)
output_test = cnn(X_test)
prediction_test = torch.argmax(output_test, axis = 1)
accuracy_test = float(prediction_test.eq(y_test).sum()) / float(y_test.shape[0])
print('Current Epoch: {}, Test Accuracy: {}'.format(epoch, accuracy_test))

Current Epoch: 999, Test Accuracy: 0.6659142212189616


In [13]:
torch.save(cnn, 'model.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(range(num_epoch), epoch_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('CNN with More Parameters')
fig.savefig('CNN_Filter_40_Size_2_Stride.png', dpi=fig.dpi)

In [15]:
epoch_loss = [i.cpu() for i in epoch_loss]
epoch_loss = [i.detach().numpy() for i in epoch_loss]

In [16]:
np.save('epoch_loss_CNN_EEG_Filter_40_Size_2_Stride.npy', epoch_loss)

In [17]:
np.save('train_acc_CNN_EEG_Filter_40_Size_2_Stride.npy', train_accuracy)

In [18]:
np.save('val_acc_CNN_EEG_Filter_40_Size_2_Stride.npy', validation_accuracy)